In [1]:
import os
import numpy as np
import pandas as pd
#from grepfunc import grep
import argparse
import modules_scima as ms
from scipy import stats
import pickle
from utils import mkdir_p
from plotting_scima import plot_results

np.random.seed(12345)

2026-01-12 20:40:13.755275: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/volumec/moritz/lib64
2026-01-12 20:40:13.755312: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


import pandas as pd

path = "/mnt/volumec/Aswath/patchmodel/s3cima/CNNMarker/fixed_embeddings256_128.csv"
out  = "/mnt/volumec/Aswath/patchmodel/s3cima/CNNMarker/fixed_embeddings256_128_clean.csv"

# Read as strings to detect header duplication safely
df = pd.read_csv(path, dtype=str)

# Detect duplicated header rows
header_cols = df.columns.tolist()

bad_rows = df.apply(
    lambda row: all(row[c] == c for c in header_cols if c.startswith("emb_")),
    axis=1
)

print(f"Removing {bad_rows.sum()} duplicated header rows")

# Remove them
df = df.loc[~bad_rows]

# Convert numeric columns
num_cols = [c for c in df.columns if c.startswith("emb_") or c in ["spatial_dim0", "spatial_dim1"]]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="raise")

# Save clean file
df.to_csv(out, index=False)

print("Clean file written to:", out)

In [2]:
# Save original output
import sys
ORIGINAL_STDOUT = sys.stdout


# Load data
Data = pd.read_csv("/mnt/volumec/Aswath/patchmodel/s3cima/contrastlearningresnet/arcsin128markerawareadaptercontrastive.csv")

Data


,cell_index,Patient,Tissue,Class0,cx,cy,emb_0,emb_1,emb_2,emb_3,...,emb_118,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127
0,1052,LHCC35,normalLiver,Hepatocytes,4941.0,2.5,-0.016557,0.003907,0.091052,-0.153683,...,0.089391,-0.056115,0.090767,-0.016186,-0.022340,0.075917,-0.116945,0.082225,-0.061024,0.058060
1,1053,LHCC35,normalLiver,Hepatocytes,5002.5,8.5,0.077811,-0.050400,0.085051,-0.176756,...,0.041904,-0.045759,0.098890,0.050759,-0.010444,0.035102,-0.117622,0.091166,0.002317,0.126521
2,1054,LHCC35,normalLiver,Hepatocytes,5031.0,9.0,0.106514,-0.059983,0.071877,-0.199953,...,0.044474,-0.071169,0.107780,0.079334,0.008188,0.042227,-0.081867,0.075300,0.058173,0.140986
3,1055,LHCC35,normalLiver,NK Cells CD16,4976.0,8.5,-0.038960,0.016652,0.047329,-0.156915,...,0.073812,-0.051692,0.103172,-0.059208,0.049716,0.030288,-0.074564,0.039946,-0.083624,0.066847
4,1056,LHCC35,normalLiver,Hepatocytes,4916.5,17.5,0.024905,-0.006658,0.069782,-0.137718,...,0.074763,-0.037562,0.115402,0.026681,-0.000330,0.079825,-0.096372,0.092575,-0.040762,0.082234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772090,375710,LHCC48,core,Tumor cells,19707.5,25192.5,0.111853,-0.037394,0.047175,-0.124155,...,0.009918,-0.035679,0.059851,0.130385,-0.099912,0.126223,-0.064698,0.098806,0.027063,0.127258
772091,375711,LHCC48,core,MHCII APCs,19934.0,25187.0,0.110738,-0.043898,0.055809,-0.129049,...,0.019920,-0.024134,0.103004,0.149812,-0.121045,0.129144,-0.079418,0.086797,0.038112,0.107882
772092,375712,LHCC48,core,Tumor cells,19278.0,25190.0,0.091269,-0.031667,0.070198,-0.132149,...,0.046251,-0.019178,0.096636,0.135823,-0.082248,0.113817,-0.093162,0.079748,0.012671,0.098426
772093,375713,LHCC48,core,Endothelial cells,19910.5,25190.0,0.112851,-0.036869,0.065368,-0.140853,...,0.019974,-0.018910,0.100959,0.133282,-0.111466,0.115471,-0.088974,0.084963,0.036458,0.099860


In [3]:

# Rename columns
Data = Data.rename(columns={
    'cx': 'spatial_dim0',
    'cy': 'spatial_dim1',
    'Class0': 'cell_type',
    'Tissue': 'condition',
    'Patient': 'sample'
})

# Columns 
cols = Data.columns.tolist()
# Drop L_variable and L_fixed columns
#Data = Data.drop(columns=['L_variable', 'L_fixed'])
Data

,cell_index,sample,condition,cell_type,spatial_dim0,spatial_dim1,emb_0,emb_1,emb_2,emb_3,...,emb_118,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127
0,1052,LHCC35,normalLiver,Hepatocytes,4941.0,2.5,-0.016557,0.003907,0.091052,-0.153683,...,0.089391,-0.056115,0.090767,-0.016186,-0.022340,0.075917,-0.116945,0.082225,-0.061024,0.058060
1,1053,LHCC35,normalLiver,Hepatocytes,5002.5,8.5,0.077811,-0.050400,0.085051,-0.176756,...,0.041904,-0.045759,0.098890,0.050759,-0.010444,0.035102,-0.117622,0.091166,0.002317,0.126521
2,1054,LHCC35,normalLiver,Hepatocytes,5031.0,9.0,0.106514,-0.059983,0.071877,-0.199953,...,0.044474,-0.071169,0.107780,0.079334,0.008188,0.042227,-0.081867,0.075300,0.058173,0.140986
3,1055,LHCC35,normalLiver,NK Cells CD16,4976.0,8.5,-0.038960,0.016652,0.047329,-0.156915,...,0.073812,-0.051692,0.103172,-0.059208,0.049716,0.030288,-0.074564,0.039946,-0.083624,0.066847
4,1056,LHCC35,normalLiver,Hepatocytes,4916.5,17.5,0.024905,-0.006658,0.069782,-0.137718,...,0.074763,-0.037562,0.115402,0.026681,-0.000330,0.079825,-0.096372,0.092575,-0.040762,0.082234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772090,375710,LHCC48,core,Tumor cells,19707.5,25192.5,0.111853,-0.037394,0.047175,-0.124155,...,0.009918,-0.035679,0.059851,0.130385,-0.099912,0.126223,-0.064698,0.098806,0.027063,0.127258
772091,375711,LHCC48,core,MHCII APCs,19934.0,25187.0,0.110738,-0.043898,0.055809,-0.129049,...,0.019920,-0.024134,0.103004,0.149812,-0.121045,0.129144,-0.079418,0.086797,0.038112,0.107882
772092,375712,LHCC48,core,Tumor cells,19278.0,25190.0,0.091269,-0.031667,0.070198,-0.132149,...,0.046251,-0.019178,0.096636,0.135823,-0.082248,0.113817,-0.093162,0.079748,0.012671,0.098426
772093,375713,LHCC48,core,Endothelial cells,19910.5,25190.0,0.112851,-0.036869,0.065368,-0.140853,...,0.019974,-0.018910,0.100959,0.133282,-0.111466,0.115471,-0.088974,0.084963,0.036458,0.099860


In [8]:
# Select only embedding columns
Data[[col for col in Data.columns if col.startswith('emb_')]]


,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_246,emb_247,emb_248,emb_249,emb_250,emb_251,emb_252,emb_253,emb_254,emb_255
0,1.758784,-0.088552,0.261380,0.865134,-0.144157,-0.000003,-0.168889,0.795359,-0.161651,-0.004781,...,-0.069732,-0.003218,1.232710,0.834299,0.485404,2.598847,1.164678,0.814252,1.006368,-0.013236
1,2.632256,0.320375,0.738415,1.000689,-0.053407,-0.000000,-0.084164,0.212483,0.059125,-0.000965,...,-0.089516,-0.021636,0.457766,0.177248,1.719815,2.161893,1.508128,0.085101,3.059182,-0.000448
2,3.538167,2.355824,1.799472,0.218069,-0.057842,-0.000051,-0.142076,0.070295,0.274163,-0.056981,...,-0.055192,-0.116194,-0.144278,0.143012,2.118090,1.128475,2.562718,1.070566,3.513464,-0.136385
3,1.760522,0.167452,1.849500,-0.007107,2.292539,-0.000000,-0.159651,-0.168656,0.201293,-0.002750,...,0.043536,-0.039637,-0.056273,-0.120167,3.517920,0.295779,0.636611,-0.120558,1.030482,0.743877
4,2.626814,1.904479,1.375503,-0.152854,0.935364,-0.000000,-0.145191,-0.168994,-0.042876,-0.001392,...,1.265651,0.032375,-0.127293,0.466546,3.910255,-0.164815,0.596795,0.557505,1.035365,-0.156836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772090,-0.151225,2.643908,0.156421,-0.159744,-0.128311,0.199237,-0.149641,2.512428,-0.094058,-0.054200,...,-0.158914,-0.068500,0.305774,0.031864,-0.169648,-0.119218,0.049787,-0.151538,0.044122,0.052121
772091,-0.097476,2.859248,0.508458,-0.121657,-0.060120,-0.169528,-0.101540,3.051634,-0.076760,0.343731,...,0.183742,-0.028442,-0.100167,-0.140990,0.261686,-0.167953,0.033243,-0.112023,0.589598,-0.013157
772092,0.289819,2.635084,0.025294,-0.079980,-0.036914,-0.035043,0.917864,2.267171,-0.147977,0.577092,...,0.377321,-0.122686,0.095779,1.351806,-0.158710,-0.168699,0.681397,-0.071014,-0.153234,-0.066336
772093,-0.048524,0.011973,0.508573,-0.169187,-0.054825,-0.054455,-0.156716,2.818503,-0.010866,0.478652,...,-0.091764,-0.041676,-0.168169,0.540946,0.520457,-0.000003,0.476733,-0.041990,0.875432,-0.086205


In [10]:
# Metadata columns
meta_cols = [
    'cell_index', 'sample', 'spatial_dim0', 'spatial_dim1', 'condition',
    'cell_type', 'patch_size','L_variable', 'L_fixed','L_raw','L_used','logL','L_over_out','pad_frac','Unnamed: 0', 'X', 'index', 'Object.Id',
]

# Marker columns = everything that is NOT metadata
#marker_cols = [c for c in cols if c not in meta_cols]

marker_cols = [c for c in Data.columns if c.startswith('emb_')]
Data[marker_cols].dtypes


emb_0      float64
emb_1      float64
emb_2      float64
emb_3      float64
emb_4      float64
            ...   
emb_251    float64
emb_252    float64
emb_253    float64
emb_254    float64
emb_255    float64
Length: 256, dtype: object

In [11]:
Intensity = Data[marker_cols].astype(float).values 
Intensity 

array([[ 1.75878420e+00, -8.85516550e-02,  2.61380140e-01, ...,
         8.14252100e-01,  1.00636830e+00, -1.32357115e-02],
       [ 2.63225650e+00,  3.20375140e-01,  7.38415200e-01, ...,
         8.51010400e-02,  3.05918200e+00, -4.47893500e-04],
       [ 3.53816680e+00,  2.35582380e+00,  1.79947160e+00, ...,
         1.07056600e+00,  3.51346450e+00, -1.36385430e-01],
       ...,
       [ 2.89819420e-01,  2.63508400e+00,  2.52935530e-02, ...,
        -7.10138700e-02, -1.53233830e-01, -6.63362700e-02],
       [-4.85237500e-02,  1.19733750e-02,  5.08573100e-01, ...,
        -4.19901460e-02,  8.75431900e-01, -8.62050900e-02],
       [ 1.15117620e-01,  1.14506670e+00,  1.26219560e-02, ...,
        -5.11546400e-02,  8.54223550e-01, -9.69033400e-02]])

In [12]:
# Create unique cell ID column
Data['cell_id'] = Data.index
# Create a unique image id column (redundant here but works with the program)
mappings = {key : idx for idx, key in enumerate(Data['sample'].unique())}
Data['image_id'] = Data['sample'].map(mappings)
Data

,Unnamed: 0,X,index,Object.Id,XMin,XMax,YMin,YMax,condition,sample,...,emb_248,emb_249,emb_250,emb_251,emb_252,emb_253,emb_254,emb_255,cell_id,image_id
0,1,0,0,0,7675,7702,-5,16,normalLiver,LHCC35,...,1.232710,0.834299,0.485404,2.598847,1.164678,0.814252,1.006368,-0.013236,0,0
1,2,1,1,1,7703,7722,-5,8,normalLiver,LHCC35,...,0.457766,0.177248,1.719815,2.161893,1.508128,0.085101,3.059182,-0.000448,1,0
2,3,2,2,2,7752,7772,-5,14,normalLiver,LHCC35,...,-0.144278,0.143012,2.118090,1.128475,2.562718,1.070566,3.513464,-0.136385,2,0
3,4,3,3,3,7828,7852,-5,12,normalLiver,LHCC35,...,-0.056273,-0.120167,3.517920,0.295779,0.636611,-0.120558,1.030482,0.743877,3,0
4,5,4,4,4,7926,7953,-5,17,normalLiver,LHCC35,...,-0.127293,0.466546,3.910255,-0.164815,0.596795,0.557505,1.035365,-0.156836,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772090,2749812,2749811,2805436,35253,15340,15359,11485,11509,rim,LHCC45,...,0.305774,0.031864,-0.169648,-0.119218,0.049787,-0.151538,0.044122,0.052121,772090,2
772091,2749813,2749812,2805437,35254,15395,15414,11485,11505,rim,LHCC45,...,-0.100167,-0.140990,0.261686,-0.167953,0.033243,-0.112023,0.589598,-0.013157,772091,2
772092,2749814,2749813,2805438,35255,15443,15457,11487,11500,rim,LHCC45,...,0.095779,1.351806,-0.158710,-0.168699,0.681397,-0.071014,-0.153234,-0.066336,772092,2
772093,2749815,2749814,2805439,35256,15214,15232,11490,11507,rim,LHCC45,...,-0.168169,0.540946,0.520457,-0.000003,0.476733,-0.041990,0.875432,-0.086205,772093,2


In [13]:
Data['cell_type'].value_counts()



cell_type
Hepatocytes                                 242274
Tumor cells                                 103606
LSECs                                        99358
Endothelial cells                            85654
T cells CD4                                  34359
Mixed immune CD45-                           22339
Biliary Epithelial cells                     19623
Vascular smooth muscle cells/Fibroblasts     17956
Granulocytes CD38-                           15201
MHCII APCs                                   14412
M2 Macrophages PD-L1+                        13225
Mixed                                        11718
M2 Macrophages PD-L1-                        11226
T cells CD8 PD-L1+                            9796
MAITs                                         9774
T cells CD8 PD-1low                           9693
T cells CD4 naïve                             9606
T cells CD8 PD-1high                          9465
Unknown                                       7754
NK Cells CD16        

In [ ]:
# Patient id
pat = np.asarray(Data['sample'])

# Seems to be the image id (spot = image in CODEX)
# In this case 1 image per patient
image = np.asarray(Data['image_id'])

# Cluster or cell type
ct = np.asarray(Data['cell_type'])

# X spatial coords
x = np.asarray(Data['spatial_dim0'])

# Y spatial coords
y = np.asarray(Data['spatial_dim1'])

# Unique cell id
cellid = np.asarray(Data['cell_id'])

# Groups to compare
groups = np.asarray(Data['condition'])

# All labels
labels = np.unique(np.asarray(Data['condition']))


In [15]:
# Output path
path =  '/mnt/volumec/Aswath/patchmodel/s3cima/markerembeddings128fixedarcsin/resultsMAIT'


#number of random sets
N = 500
#number of nearest neighbor cells 
K = 50
Anchor = 'BG'
OUTDIR = path + '/Anchor' + str(Anchor) + f'_K{K}' 

for i in range(0, len(labels)):
    inx = np.where(groups == labels[i])[0]
    rd, rs, nset = ms.BG_spatial_input_per_sample(N, image[inx], pat[inx], Intensity[inx,:], ct[inx], x[inx], 
                               y[inx], cellid[inx], K, labels[i], OUTDIR)


NameError: name 'labels' is not defined

In [10]:
import modules_scima as ms

path =  '/mnt/volumec/Aswath/patchmodel/s3cima/CNNResnetembeddings128fixedarcsin/resultsMAIT'


# Hyperparameters
Anchor = 'BG'
ntrain_per_class = 2
K = 50 
k = 50
nrun = 50
nset_thr = 1
background = False

# Label mapping
labels = ['core', 'normalLiver', 'rim']
classes = [0, 1,2]




ms.run_scima(Anchor, ntrain_per_class, K, k, nset_thr, labels, classes, path, nrun, background)

output will be redirected to /mnt/volumec/Aswath/patchmodel/s3cima/CNNResnetembeddings128fixedarcsin/resultsMAIT/resultsBG_K50/run_log//data_run_log_10_01_2026_07_07_04.txt


2026-01-10 07:07:09.349100: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-01-10 07:07:09.349566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-01-10 07:07:09.349914: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/volumec/moritz/lib64
2026-01-10 07:07:09.349997: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/volumec/moritz/lib64
2026-01-10 07:07

KeyError: 49

In [ ]:
path =  '/mnt/volumec/Aswath/patchmodel/s3cima/CNNResnetembeddings128fixedarcsin/resultsMAIT'


#number of nearest neighbor cells around each anchor cell
K = 50

CTs = np.unique(np.asarray(Data['cell_type']))
Anchor = 'MAITs'
OUTDIR = path + '/Anchor' + str(Anchor) + f'_K{K}' 



for i in range(0, len(labels)):
    inx = np.where(groups == labels[i])[0]
    d,s,rd,rs,nset = ms.spatial_input_per_sample(Anchor,image[inx], pat[inx], Intensity[inx,:], ct[inx], 
                                                  x[inx], y[inx], cellid[inx], K, labels[i], OUTDIR)

In [ ]:
Anchor = 'MAITs'
K = 50
ntrain_per_class = 2
k = 10

# if background sets used the order should be first the sample labels and the random set at the end. For example here,
# the background label is 2.

labels = ['core', 'normalLiver','rim','randcore','randnormalLiver','randrim']
classes = [0, 1,2,0,1,2]

background = True 

path =  '/mnt/volumec/Aswath/patchmodel/s3cima/CNNResnetembeddings128fixedarcsin/resultsMAIT'

print("Unique regions in S3-CIMA input:", np.unique(groups))
nrun = 50
nset_thr = 0.9

print("\n=== MARKER COLUMNS PASSED TO SCIMa ===")
print(len(marker_cols), marker_cols)
print("Intensity shape:", Intensity.shape)

ms.run_scima(Anchor, ntrain_per_class, K, k, nset_thr, labels, classes, path, nrun, background)

Unique regions in S3-CIMA input: ['core' 'normalLiver' 'rim']

=== MARKER COLUMNS PASSED TO SCIMa ===
512 ['emb_0', 'emb_1', 'emb_2', 'emb_3', 'emb_4', 'emb_5', 'emb_6', 'emb_7', 'emb_8', 'emb_9', 'emb_10', 'emb_11', 'emb_12', 'emb_13', 'emb_14', 'emb_15', 'emb_16', 'emb_17', 'emb_18', 'emb_19', 'emb_20', 'emb_21', 'emb_22', 'emb_23', 'emb_24', 'emb_25', 'emb_26', 'emb_27', 'emb_28', 'emb_29', 'emb_30', 'emb_31', 'emb_32', 'emb_33', 'emb_34', 'emb_35', 'emb_36', 'emb_37', 'emb_38', 'emb_39', 'emb_40', 'emb_41', 'emb_42', 'emb_43', 'emb_44', 'emb_45', 'emb_46', 'emb_47', 'emb_48', 'emb_49', 'emb_50', 'emb_51', 'emb_52', 'emb_53', 'emb_54', 'emb_55', 'emb_56', 'emb_57', 'emb_58', 'emb_59', 'emb_60', 'emb_61', 'emb_62', 'emb_63', 'emb_64', 'emb_65', 'emb_66', 'emb_67', 'emb_68', 'emb_69', 'emb_70', 'emb_71', 'emb_72', 'emb_73', 'emb_74', 'emb_75', 'emb_76', 'emb_77', 'emb_78', 'emb_79', 'emb_80', 'emb_81', 'emb_82', 'emb_83', 'emb_84', 'emb_85', 'emb_86', 'emb_87', 'emb_88', 'emb_89', 'em